## Cohere RAG chatbot with document embedding with gardio ui

In [ ]:
!pip install langchain
!pip install pypdf
!pip install cohere
!pip install qdrant-client
!pip install gradio
!pip install langchain-community
!pip install sentence-transformers
!pip install faiss-cpu

In [ ]:
#Import Libraries
import os
import langchain
import gradio as gr


In [ ]:
os.environ['COHERE_API_KEY'] =  ''    #Set the Cohere API Key

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
DATA_PATH = "./"
DB_FAISS_PATH = "./"

loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                chunk_overlap=50)
texts = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                    model_kwargs={'device': 'cpu'})

db = FAISS.from_documents(texts, embeddings)
db.save_local(DB_FAISS_PATH)

In [ ]:
from langchain import PromptTemplate

In [ ]:
"""
Prompt template for QA retrieval for each vectorstore
"""
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])


In [ ]:
import langchain
langchain.debug = False
from langchain.chains import RetrievalQA
from langchain.llms import Cohere
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0.9),
                                 chain_type="stuff",
                                 retriever=db.as_retriever(),
                                 chain_type_kwargs={'prompt': prompt},
                                 return_source_documents=True)

In [ ]:
questions = [
           "what are the variants of senior first ?",
           "pre hospitalization ?",
           "post hospitalization ?",
           "Entry age?"
           ]

In [ ]:
for question in questions:
  answer = qa({"query": question})
  print(answer)

### Document converse bot

In [ ]:
from langchain.llms import Cohere
llm = Cohere(model = "command", temperature=0.9)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
#Setup Chat History
#The chat_history variable keeps track of the conversation history,
#storing the user queries #and the chatbot's responses
#This allows the chatbot to maintain context and provide relevant answers
# based on past interactions.
chat_history = []
qa = ConversationalRetrievalChain.from_llm(
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = db.as_retriever(),
    return_source_documents=False,
    combine_docs_chain_kwargs={'prompt': prompt}
)

In [ ]:
def QandA(question):
    global chat_history
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    return result['answer']

In [ ]:
#DocumentQA Interface
import gradio as gr
DocQABotUI = gr.Interface(
    fn=QandA,
    inputs="text",
    outputs="text",
    title="Chatbot",
    description="Document QA"
)
#Launching the DocQABot
DocQABotUI.launch(share=True)